# Forecasting with XGBOOST

**Introduction/Methodology/DataDict**

Methodology:
A Random Forest Model will be used to try to forecast wave height and wave period. First a dataframe will be created, containing lagged features as well as cyclic encoded variables for month season and week. 
Moon phase added 

In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import decimal
from datetime import datetime

In [2]:
df=pd.read_csv('../Data/df_daily_imputed.csv', index_col=0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9902 entries, 1988-11-22 to 2016-01-01
Data columns (total 33 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LATITUDE   9902 non-null   float64
 1   LONGITUDE  9902 non-null   float64
 2   DEPTH      9902 non-null   float64
 3   VWH$       9902 non-null   float64
 4   VCMX       9902 non-null   float64
 5   VTP$       9902 non-null   float64
 6   WDIR       9902 non-null   float64
 7   WSPD       9902 non-null   float64
 8   GSPD       9902 non-null   float64
 9   WDIR.1     9902 non-null   float64
 10  WSPD.1     9902 non-null   float64
 11  GSPD.1     9902 non-null   float64
 12  ATMS       9902 non-null   float64
 13  DRYT       9902 non-null   float64
 14  SSTP       9902 non-null   float64
 15  YEAR       9902 non-null   float64
 16  WD         9902 non-null   float64
 17  WS         9902 non-null   float64
 18  ETOT       9902 non-null   float64
 19  TP         9902 non-null   float64
 20

## Feature Engineering

In [4]:
# convert directions(degrees North) into radians
columns_to_convert = ['VMD', 'VMDSea', 'VMDSw', 'WD', 'WDIR', 'WDIR.1']

# Convert specified columns to radians
df[columns_to_convert] = np.radians(df[columns_to_convert])

In [6]:
# Define lags for different time intervals
lags = {'1_day': 1, '1_week': 7, '1_month': 30, '3_month': 90}


# Create a new DataFrame to avoid modifying the original DataFrame in place
new_df = pd.DataFrame()

# Create lags, rolling averages, and std deviations for all numeric columns
for column in df.select_dtypes(include='number').columns:
    # Create lags for different time intervalsa
    for lag_name, lag_value in lags.items():
        new_df[f'{column}_lag_{lag_name}'] = df[column].shift(lag_value)


# Combine the new features with the original DataFrame
features_df = pd.concat([df, new_df], axis=1)

# Drop rows with null values
features_df = features_df.dropna()

# Display the modified DataFrame
print(features_df.head())


               LATITUDE  LONGITUDE  DEPTH      VWH$      VCMX       VTP$  \
Datetime_buoy                                                              
1989-02-20        48.83      126.0   73.0  2.000417  3.683333  12.922500   
1989-02-21        48.83      126.0   73.0  2.281739  3.926087  13.330435   
1989-02-22        48.83      126.0   73.0  2.645000  4.691667  10.620833   
1989-02-23        48.83      126.0   73.0  2.488750  4.337500  10.010000   
1989-02-24        48.83      126.0   73.0  2.564583  4.475000  12.950833   

                   WDIR       WSPD       GSPD    WDIR.1  ...  \
Datetime_buoy                                            ...   
1989-02-20     2.070397   9.537500  11.558333  1.914772  ...   
1989-02-21     2.057971  10.847826  13.182609  1.891026  ...   
1989-02-22     2.600541   8.416667  10.287500  2.431098  ...   
1989-02-23     2.953970   5.193750   6.987500  2.781618  ...   
1989-02-24     1.845686   5.983333   7.516667  1.693697  ...   

               DMD

/var/folders/h5/pnqm5hvd2vj1397_ck_gmtj00000gn/T/ipykernel_18479/3457223367.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[f'{column}_lag_{lag_name}'] = df[column].shift(lag_value)
/var/folders/h5/pnqm5hvd2vj1397_ck_gmtj00000gn/T/ipykernel_18479/3457223367.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[f'{column}_lag_{lag_name}'] = df[column].shift(lag_value)
/var/folders/h5/pnqm5hvd2vj1397_ck_gmtj00000gn/T/ipykernel_18479/3457223367.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [7]:
features_df.shape

(9812, 165)

In [9]:
features_df.index = pd.to_datetime(features_df.index)

In [11]:
features_df.head(1)

,LATITUDE,LONGITUDE,DEPTH,VWH$,VCMX,VTP$,WDIR,WSPD,GSPD,WDIR.1,...,ANGSPR_lag_1_month,ANGSPR_lag_3_month,INLINE_lag_1_day,INLINE_lag_1_week,INLINE_lag_1_month,INLINE_lag_3_month,month_sin,month_cos,season_sin,season_cos
Datetime_buoy,,,,,,,,,,,,,,,,,,,,,
1989-02-20,48.83,126.0,73.0,2.000417,3.683333,12.9225,2.070397,9.5375,11.558333,1.914772,...,0.73862,0.8075,0.730675,0.660987,0.727566,0.7153,0.866025,0.5,0.14112,0.989992


**Cyclic Encode Temporal Features**

In [15]:
#Create cyclical encoded features for month, season, and week
features_df['month_sin'] = np.sin(2 * np.pi * features_df.index.month / 12)
features_df['month_cos'] = np.cos(2 * np.pi * features_df.index.month / 12)

#Assume seasons are defined as quarters (1-4)
features_df['season_sin'] = np.sin(2 * np.pi * features_df.index.month % 12 / 4)
features_df['season_cos'] = np.cos(2 * np.pi * features_df.index.month % 12 / 4)

features_df['week_sin'] = np.sin(2 * np.pi * features_df.index.strftime('%U').astype(int) / 52)  # Assuming 52 weeks in a year
features_df['week_cos'] = np.cos(2 * np.pi * features_df.index.strftime('%U').astype(int) / 52)


/var/folders/h5/pnqm5hvd2vj1397_ck_gmtj00000gn/T/ipykernel_18479/5589926.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df['week_sin'] = np.sin(2 * np.pi * features_df.index.strftime('%U').astype(int) / 52)  # Assuming 52 weeks in a year
/var/folders/h5/pnqm5hvd2vj1397_ck_gmtj00000gn/T/ipykernel_18479/5589926.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df['week_cos'] = np.cos(2 * np.pi * features_df.index.strftime('%U').astype(int) / 52)


In [18]:
features_df.shape

(9812, 171)

In [24]:
# add moonphase as a column (Code for moonphase taken from kaggle: https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion/154776)
def get_moon_phase(d):  # 0=new, 4=full; 4 days/phase
    diff = d - datetime(2001, 1, 1)
    days = decimal.Decimal(diff.days) + (decimal.Decimal(diff.seconds) / decimal.Decimal(86400))
    lunations = decimal.Decimal("0.20439731") + (days * decimal.Decimal("0.03386319269"))
    phase_index = math.floor((lunations % decimal.Decimal(1) * decimal.Decimal(8)) + decimal.Decimal('0.5'))
    return int(phase_index) & 7

In [25]:
features_df['moon_phase'] = features_df.index.map(get_moon_phase)

In [28]:
#cyclic encode the moonphase as it is ordinal, then drop moon phase
features_df['moon_phase_sin'] = np.sin(2 * np.pi * features_df['moon_phase'] / 8)
features_df['moon_phase_cos'] = np.cos(2 * np.pi * features_df['moon_phase'] / 8)

/var/folders/h5/pnqm5hvd2vj1397_ck_gmtj00000gn/T/ipykernel_18479/2594305688.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df['moon_phase_sin'] = np.sin(2 * np.pi * features_df['moon_phase'] / 8)
/var/folders/h5/pnqm5hvd2vj1397_ck_gmtj00000gn/T/ipykernel_18479/2594305688.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df['moon_phase_cos'] = np.cos(2 * np.pi * features_df['moon_phase'] / 8)


In [31]:
features_df=features_df.drop('moon_phase', axis=1)

In [32]:
features_df.shape

(9812, 173)

## Modelling

In [64]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import time
from sklearn.metrics import make_scorer


**Train Test Split**

First a test train split will be done, on the datetime index. Since datetime index is in order 80% of the len of the data frame will be taken in order for the train set. TimeSeries Split will be used for cross validation from sklearn.

In [38]:
#split on date for train and test
split_point = '2006-01-01'
#filter data on split point
train_xg= features_df.index < split_point
test_xg = features_df.index >= split_point

In [39]:
#define X and y
X_train = features_df[train_xg].drop(['VTP$'],axis=1)
y_train = features_df[train_xg]['VTP$']

X_test = features_df[test_xg].drop(['VTP$'], axis=1)
y_test = features_df[test_xg]['VTP$']

In [40]:
X_train.shape

(6159, 172)

In [41]:
X_test.shape

(3653, 172)

**Pipeline**

 **System Specifications and Parallelization**

- **Model Name:** Mac mini
- **Chip:** Apple M2
- **Total Number of Cores:** 8 (4 performance and 4 efficiency)
- **Memory:** 16 GB


**Parallization**
n_jobs = 3


In [42]:
#pipeline object
xgb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb_model',xgb.XGBRegressor(objective ='reg:squarederror',random_state=42))
])

In [54]:
#Param Grid
param_grid_gbtree = {
    'xgb_model__booster': ['gbtree'],
    'xgb_model__learning_rate': [0.01, 0.1, 0.2],
    'xgb_model__max_depth': [3, 5, 7],
    'xgb_model__n_estimators': [50, 100, 200],
}

# Param grid for gblinear booster
param_grid_gblinear = {
    'xgb_model__booster': ['gblinear'],
    'xgb_model__learning_rate': [0.01, 0.1, 0.2],
    'xgb_model__reg_alpha': [0, 0.1, 0.5],
}


param_grid =[param_grid_gbtree, param_grid_gblinear]

In [55]:
#grid search
#cv = sklearn TimeSeries Split
tscv = TimeSeriesSplit(n_splits=3)
#set timer
start_time = time.time()
grid_search = GridSearchCV(xgb_pipeline, param_grid, cv=tscv, scoring='neg_mean_squared_error',n_jobs=3) #will optimize for smallest mse
grid_search.fit(X_train, y_train)
#end timer
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Grid search completed in {elapsed_time:.2f} seconds.")

/Users/kamiasalango/anaconda3/envs/capstone_wavepower/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Grid search completed in 41.14 seconds.


In [56]:
#get the best params
best_params = grid_search.best_params_
print('Best Hyperparameters:')
print(best_params)

Best Hyperparameters:
{'xgb_model__booster': 'gbtree', 'xgb_model__learning_rate': 0.2, 'xgb_model__max_depth': 3, 'xgb_model__n_estimators': 50}


In [69]:
results_list =[]

Hyperparameters are at boundaries of ranges. Grid Search will be run again with an expanded hyperparameter space and just on gbtree as it was the best model. In future may come back to gblinear and try to optimize. 

In [71]:
#Use function for grid search
def run_grid_search(X_train, y_train, param_grid, pipeline, scoring_metric, identifier):
    """
    Run a grid search with the specified parameters.

    Parameters:
    - X_train: Training features
    - y_train: Training labels
    - param_grid: Parameter grid for the grid search
    - pipeline: pipeline object
    - scoring_metric: Scikit-learn scoring metric
    - identifier: Identifier for iteration of Gridsearch'

    Returns:
    - grid_search: Fitted GridSearchCV object 
    -resluts of cross validation in terms of best params, best score (validation set)
    - time it took to run GridSearch
    """
    #initiate timer module
    start_time = time.time()
    #time series cv
    tscv = TimeSeriesSplit(n_splits=3)
    # Set up the scoring metric
    scoring = scoring_metric

    # Instantiate GridSearchCV with 5-fold cross-validation, n_jobs=3, and specified scoring metric
    grid_search = GridSearchCV(pipeline, param_grid, cv=tscv, scoring=scoring, n_jobs=3)

    # Fit and run grid search
    grid_search.fit(X_train, y_train)

    #end timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    
     # Store the results (hyperparameters and scores)
    results_list.append({
        'identifier': identifier,
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'elapsed_time': elapsed_time
    })
    
    # Print the best parameters with the identifier
    print(f"Best Parameters for {identifier}: {grid_search.best_params_}")

    # Print the best score on the validation sets, 
    #.best_score_ is attribute of GridSearch CV that accesses best validation score(score specified in GS)
    print(f"Best {scoring_metric} Score for {identifier}: {grid_search.best_score_}")
    # Print the elapsed time
    print(f"Elapsed Time for {identifier}: {elapsed_time} seconds")

    return grid_search


**GridSearch run, Identifier = optimize_tree**

In [58]:
#run gridsearch with expanded params: 
param_grid_gbtree = {
    'xgb_model__booster': ['gbtree'],
    'xgb_model__learning_rate': [0.01, 0.1, 0.2, 0.5],  # Added 0.5
    'xgb_model__max_depth': [2,3, 5, 7, 10],  # Added 2, 10
    'xgb_model__n_estimators': [50, 100, 200, 300],  # Added 300
    'xgb_model__subsample': [0.8, 0.9, 1.0],  # Add subsample
    'xgb_model__colsample_bytree': [0.8, 0.9, 1.0],  # Add colsample_bytree
}

In [ ]:
scoring_metric = 'neg_mean_squared_error'
run_grid_search(X_train, y_train, param_grid_gbtree, xgb_pipeline, scoring_metric, 'optimize_tree')

/Users/kamiasalango/anaconda3/envs/capstone_wavepower/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
